## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Bambous Virieux,MU,-20.3428,57.7575,69.22,83,20,13.80,light rain,2021-08-18 00:17:58
1,1,Saldanha,ZA,-33.0117,17.9442,55.45,89,100,9.08,overcast clouds,2021-08-18 00:17:59
2,2,Avarua,CK,-21.2078,-159.7750,75.25,60,75,13.80,broken clouds,2021-08-18 00:17:59
3,3,Yellowknife,CA,62.4560,-114.3525,59.52,55,20,11.50,few clouds,2021-08-18 00:17:36
4,4,Busselton,AU,-33.6500,115.3333,56.08,85,4,3.96,clear sky,2021-08-18 00:18:00


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                   (city_data_df["Max Temp"] >= min_temp)]
preferred_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
2,2,Avarua,CK,-21.2078,-159.7750,75.25,60,75,13.80,broken clouds,2021-08-18 00:17:59
8,8,Guapi,CO,2.5708,-77.8854,76.86,87,70,3.27,broken clouds,2021-08-18 00:11:18
9,9,Butaritari,KI,3.0707,172.7902,83.44,69,92,12.95,overcast clouds,2021-08-18 00:18:01
10,10,Santarem,BR,-2.4431,-54.7083,81.91,60,62,1.66,broken clouds,2021-08-18 00:11:19
14,14,Kapaa,US,22.0752,-159.3190,84.52,67,40,1.01,scattered clouds,2021-08-18 00:18:02


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities.count()

City_ID                249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
Date                   249
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities = preferred_cities.dropna()
preferred_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
2,2,Avarua,CK,-21.2078,-159.7750,75.25,60,75,13.80,broken clouds,2021-08-18 00:17:59
8,8,Guapi,CO,2.5708,-77.8854,76.86,87,70,3.27,broken clouds,2021-08-18 00:11:18
9,9,Butaritari,KI,3.0707,172.7902,83.44,69,92,12.95,overcast clouds,2021-08-18 00:18:01
10,10,Santarem,BR,-2.4431,-54.7083,81.91,60,62,1.66,broken clouds,2021-08-18 00:11:19
14,14,Kapaa,US,22.0752,-159.3190,84.52,67,40,1.01,scattered clouds,2021-08-18 00:18:02


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,
8,Guapi,CO,76.86,broken clouds,2.5708,-77.8854,
9,Butaritari,KI,83.44,overcast clouds,3.0707,172.7902,
10,Santarem,BR,81.91,broken clouds,-2.4431,-54.7083,
14,Kapaa,US,84.52,scattered clouds,22.0752,-159.3190,
18,Monrovia,LR,77.63,broken clouds,6.3005,-10.7969,
19,Shahr-E Babak,IR,75.90,broken clouds,30.1165,55.1186,
21,Atuona,PF,77.29,light rain,-9.8000,-139.0333,
25,Aitape,PG,82.85,overcast clouds,-3.1374,142.3475,
29,Kahului,US,84.22,light rain,20.8947,-156.4700,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f'{lat},{lng}'
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
print("Hotels located.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotels located.


In [10]:
hotel_df.count()

City                   249
Country                249
Max Temp               249
Current Description    249
Lat                    249
Lng                    249
Hotel Name             249
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,Paradise Inn
8,Guapi,CO,76.86,broken clouds,2.5708,-77.8854,Hotel Emperador Guapí
9,Butaritari,KI,83.44,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
10,Santarem,BR,81.91,broken clouds,-2.4431,-54.7083,Ecotur Lodge
14,Kapaa,US,84.52,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
18,Monrovia,LR,77.63,broken clouds,6.3005,-10.7969,Sunset Inn
19,Shahr-E Babak,IR,75.90,broken clouds,30.1165,55.1186,Duna Ecolodge
21,Atuona,PF,77.29,light rain,-9.8000,-139.0333,Villa Enata
25,Aitape,PG,82.85,overcast clouds,-3.1374,142.3475,
29,Kahului,US,84.22,light rain,20.8947,-156.4700,Maui Seaside Hotel


In [12]:
hotel_df["Hotel Name"]

2                             Paradise Inn
8                    Hotel Emperador Guapí
9                       Isles Sunset Lodge
10                            Ecotur Lodge
14     Sheraton Kauai Coconut Beach Resort
                      ...                 
701                         Hotel Amparito
703                    Dibbo areaمنطقة دبو
704                                       
707                   Tinidee Hotel@Ranong
710                           Delanco Camp
Name: Hotel Name, Length: 249, dtype: object

In [13]:
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,Paradise Inn
8,Guapi,CO,76.86,broken clouds,2.5708,-77.8854,Hotel Emperador Guapí
9,Butaritari,KI,83.44,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
10,Santarem,BR,81.91,broken clouds,-2.4431,-54.7083,Ecotur Lodge
14,Kapaa,US,84.52,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
18,Monrovia,LR,77.63,broken clouds,6.3005,-10.7969,Sunset Inn
19,Shahr-E Babak,IR,75.90,broken clouds,30.1165,55.1186,Duna Ecolodge
21,Atuona,PF,77.29,light rain,-9.8000,-139.0333,Villa Enata
29,Kahului,US,84.22,light rain,20.8947,-156.4700,Maui Seaside Hotel
34,Champerico,GT,85.35,broken clouds,14.3000,-91.9167,"Hotel y restaurante ""El Diamante"""


In [14]:
clean_hotel_df.count()

City                   237
Country                237
Max Temp               237
Current Description    237
Lat                    237
Lng                    237
Hotel Name             237
dtype: int64

In [22]:
# 8a. Create the output File (CSV)
hotel_output = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(hotel_output, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name:</dt><dd>{Hotel Name}</dd>
<dt>City:</dt><dd>{City}</dd>
<dt>Country:</dt><dd>{Country}</dd>
<dt>Current Weather:</dt><dd>{Current Description}</dd>
<dt>Max Temp:</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0),zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))